# LangChain Quickstart

In [1]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "[YOUR_KEY_HERE]"
os.environ["SERPAPI_API_KEY"] = "[YOUR_KEY_HERE]"

openai.api_key = "[YOUR_KEY_HERE]"

In [2]:
from langchain.callbacks import ClearMLCallbackHandler
from langchain.callbacks.base import CallbackManager

from clearml import Task


# Set up a new ClearML Task
task = Task.init(
    project_name="Promptimizer",
    task_name="Quickstart",
    output_uri=True
)


# Set up the ClearML handler
clearml_callback = ClearMLCallbackHandler(
    task_type="inference",
    project_name="langchain_callback_demo",
    task_name="llm",
    tags=["test"],
    # Change the following parameters based on the amount of detail you want tracked
    visualize=True,
    complexity_metrics=False,
    stream_logs=False
)
# manager = CallbackManager([StdOutCallbackHandler(), clearml_callback])
manager = CallbackManager([clearml_callback])

ClearML Task: created new task id=2330d905c2004835bcb63ec92da405b2
2023-04-12 11:05:30,197 - clearml.Task - INFO - Storing jupyter notebook directly as code
2023-04-12 11:05:30,201 - clearml.Repository Detection - WARNING - Can't get url information for git repo in /home/victor/Projects/langchain
ClearML results page: https://app.clear.ml/projects/5e04ba0139f044d280552b0e06ad3681/experiments/2330d905c2004835bcb63ec92da405b2/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
The clearml callback is currently in beta and is subject to change based on updates to `langchain`. Please report any issues to https://github.com/allegroai/clearml/issues with the tag `langchain`.


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f284aa426b0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/events.add_batch
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f284aa41c30>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/events.add_batch


## Simple LLM inference

In [3]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9, callback_manager=manager, verbose=True)
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))
clearml_callback.flush_tracker(langchain_asset=llm, name="sock_company")



Colorful Comfort Socks.
2023-04-12 11:07:18,995 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Promptimizer/Quickstart.2330d905c2004835bcb63ec92da405b2/models/sock_company


In [4]:
llm = OpenAI(temperature=0.1, callback_manager=manager, verbose=True)
text = "How do I get started with ClearML?"
print(llm(text))
clearml_callback.flush_tracker(langchain_asset=llm, name="clearml_getting_started")



To get started with ClearML, you can sign up for a free trial or purchase a subscription. Once you have an account, you can start using the ClearML platform to manage your machine learning experiments, track metrics, and deploy models. You can also use the ClearML Python SDK to integrate ClearML into your existing ML workflow.
2023-04-12 11:08:09,983 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Promptimizer/Quickstart.2330d905c2004835bcb63ec92da405b2/models/clearml_getting_started


## Prompt customization

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [7]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes clearml?


## Chaining

In [8]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6, callback_manager=manager, verbose=True)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?\n\n",
)

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
print(chain.run("ethically sources smartphones"))
clearml_callback.flush_tracker(langchain_asset=chain, name="ethical_chain")


EcoSmart Phones
2023-04-12 11:11:56,314 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Promptimizer/Quickstart.2330d905c2004835bcb63ec92da405b2/models/ethical_chain


# Premade chains

In [11]:
from langchain import OpenAI, LLMMathChain

llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)
llm_math = LLMMathChain(llm=llm, verbose=True)

llm_math.run("What is 13 raised to the .3432 power?")
clearml_callback.flush_tracker(langchain_asset=llm_math, name="llm_math")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?
```python
import math
print(math.pow(13, 0.3432))
```

Answer: 2.4116004626599237

> Finished chain.
2023-04-12 11:13:40,428 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Promptimizer/Quickstart.2330d905c2004835bcb63ec92da405b2/models/llm_math


## Creating an "Agent"

In [12]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("Where can I find and download ClearML?")
clearml_callback.flush_tracker(langchain_asset=agent, name="google agent")




> Entering new AgentExecutor chain...


/home/victor/Projects/langchain/.venv/lib/python3.10/site-packages/spacy/displacy/__init__.py:211: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


 I need to find a website that offers ClearML.
Action: Search
Action Input: "ClearML download"
Observation: Easily Develop, Orchestrate, and Automate ML workflows at scale. ClearML's open-source MLOps platform is your end-to-end solution. Start free!
Thought: I should visit the website to download ClearML.
Action: Search
Action Input: "ClearML website"
Observation: Easily Develop, Orchestrate, and Automate ML workflows at scale. ClearML's open-source MLOps platform is your end-to-end solution. Start free!
Thought: I should visit the website to download ClearML.
Action: Search
Action Input: "ClearML download page"
Observation: Return a base folder with a read-only (immutable) local copy of the entire dataset download and copy / soft-link, files from all the parent dataset versions ...
Thought: I should visit the website to download ClearML.
Action: Search
Action Input: "ClearML download link"
Observation: External file links can be from cloud storage (s3://, gs://, azure://), ... downlo

Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2808597430>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/tasks.get_all
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2808595c60>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/tasks.get_all
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f28085977f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/tasks.get_all


2023-04-13 07:59:56,239 - clearml.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###


: 